<a href="https://colab.research.google.com/github/hiddenntreasure/Cervical-Cancer-Screening/blob/master/Numpy_of_khairulice_of_Cervical_Cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cervical

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
import numpy as np
train_x = np.load('/content/drive/My Drive/Colab Notebooks/train.npy')
train_y = np.load('/content/drive/My Drive/Colab Notebooks/train_labels.npy')
#val_x = np.load('/content/drive/My Drive/Colab Notebooks/cervical/val.npy')
#val_y = np.load('/content/drive/My Drive/Colab Notebooks/cervical/val_labels.npy')
print(len(train_x), len(train_y))
#print(len(val_x), len(val_y))
print(train_x.shape)
print(train_y.shape)

8212 8212
(8212, 200, 200, 3)
(8212, 3)


# VGG-19 without bottlenect approach

In [4]:
from keras.applications import VGG19
conv_base = VGG19(weights='imagenet',include_top=False,input_shape=(200,200,3))

from keras.applications import nasnet
#conv_base = nasnet.NASNetMobile(weights='imagenet', include_top=False, input_shape=None)

from keras.applications.resnet import ResNet152
#conv_base = ResNet152(weights= 'imagenet', include_top=False, input_shape= (200,200,3))

from keras.applications.densenet import DenseNet201
#conv_base = DenseNet201(weights= 'imagenet', include_top=False, input_shape= (200,200,3))

Using TensorFlow backend.


In [0]:
conv_base.summary()

In [5]:
from keras import models
from keras import layers
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 6, 6, 512)         20024384  
_________________________________________________________________
flatten_1 (Flatten)          (None, 18432)             0         
Total params: 20,024,384
Trainable params: 20,024,384
Non-trainable params: 0
_________________________________________________________________


In [0]:
from keras import regularizers
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))
#conv_base.trainable = False

In [0]:
print('This is the number of trainable weights ''before freezing the conv base:', len(model.trainable_weights))
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
  if layer.name == 'conv5_block20_0_relu':
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False
print('This is the number of trainable weights ''before freezing the conv base:', len(model.trainable_weights))

This is the number of trainable weights before freezing the conv base: 606
This is the number of trainable weights before freezing the conv base: 82


In [8]:
from sklearn.model_selection import train_test_split

x_train , x_val, y_train , y_val = train_test_split(train_x,train_y,test_size=0.3,stratify=train_y)

print(len(x_train), len(y_train))
print(len(x_val), len(y_val))

5748 5748
2464 2464


In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
batch_size = 50

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
test_datagen = ImageDataGenerator(rescale=1./255)

checkpoint = ModelCheckpoint('model-{epoch:03d}-{acc:03f}-{val_acc:03f}.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')

model.compile(loss='categorical_crossentropy',optimizer=optimizers.RMSprop(lr=1e-5),metrics=['acc'])
#model.compile(optimizer='adamax', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit_generator(train_datagen.flow(x_train,y_train, batch_size=batch_size, shuffle=True),
    steps_per_epoch=5748//batch_size,
    epochs=400,
    validation_data=test_datagen.flow(x_val,y_val),
    validation_steps=2464//batch_size,
    callbacks=[checkpoint]
)

Epoch 1/400
114/114 [==============================] - 65s 574ms/step - loss: 1.0885 - acc: 0.4230 - val_loss: 1.0691 - val_acc: 0.5236

Epoch 00001: val_loss improved from inf to 1.06907, saving model to model-001-0.422955-0.523597.h5
Epoch 2/400
114/114 [==============================] - 63s 552ms/step - loss: 1.0822 - acc: 0.4461 - val_loss: 1.0459 - val_acc: 0.5249

Epoch 00002: val_loss improved from 1.06907 to 1.04594, saving model to model-002-0.446121-0.524872.h5
Epoch 3/400
114/114 [==============================] - 63s 552ms/step - loss: 1.0773 - acc: 0.4491 - val_loss: 1.0709 - val_acc: 0.5376

Epoch 00003: val_loss did not improve from 1.04594
Epoch 4/400
114/114 [==============================] - 63s 555ms/step - loss: 1.0798 - acc: 0.4600 - val_loss: 1.0337 - val_acc: 0.5261

Epoch 00004: val_loss improved from 1.04594 to 1.03373, saving model to model-004-0.459986-0.526148.h5
Epoch 5/400
114/114 [==============================] - 64s 565ms/step - loss: 1.0685 - acc: 0.46

In [0]:
print('This is the number of trainable weights ''before freezing the conv base:', len(model.trainable_weights))

In [0]:
model.save_weights("/content/drive/My Drive/Colab Notebooks/cervical/model_vgg16_200_50epoch.h5")

In [0]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()